In [3]:
from typing import Any, Dict
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")


chat = ChatOpenAI(temperature=0.1, streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()],)

prompt.format(country="Germany")

'What is the capital of Germany???'